In [1]:
import CNN_draft1
from torchsummary import summary

Using device: cpu
[1,     5] loss: 247.530
[2,     5] loss: 44.169
[3,     5] loss: 251.536
[4,     5] loss: 233.053
[5,     5] loss: 246.758
[6,     5] loss: 233.274
[7,     5] loss: 239.516
[8,     5] loss: 236.109
[9,     5] loss: 236.623
[10,     5] loss: 217.729
[11,     5] loss: 231.625
[12,     5] loss: 234.586
[13,     5] loss: 211.129
[14,     5] loss: 229.567
[15,     5] loss: 202.959
[16,     5] loss: 37.651
[17,     5] loss: 36.601
[18,     5] loss: 249.259
[19,     5] loss: 243.550
[20,     5] loss: 241.254
Finished Training
Total time run: 0.5394 minutes


In [6]:
CNN_draft1

<module 'CNN_draft1' from '/Users/katie/queens_bio/NE_project/NEvolve/CNN/CNN_draft1.py'>

In [3]:
%run 'CNN_draft1.py'

Using device: cpu
[1,     5] loss: 230.235
[2,     5] loss: 254.011
[3,     5] loss: 10.818
[4,     5] loss: 38.728
[5,     5] loss: 211.808
[6,     5] loss: 231.649
[7,     5] loss: 251.610
[8,     5] loss: 64.385
[9,     5] loss: 212.570
[10,     5] loss: 256.505
[11,     5] loss: 212.618
[12,     5] loss: 256.107
[13,     5] loss: 238.783
[14,     5] loss: 236.976
[15,     5] loss: 63.855
[16,     5] loss: 255.372
[17,     5] loss: 254.802
[18,     5] loss: 237.852
[19,     5] loss: 230.351
[20,     5] loss: 52.337
Finished Training
Total time run: 0.5378 minutes


In [4]:
model = Net()
model.load_state_dict(torch.load('./test.pth'))

<All keys matched successfully>

In [5]:
with np.load('./toy_dat/sliced_data.npz') as data:
    param = data['param']
    raw_sim = data['sim']
    pos = data['pos']
	
# Normalize to mean of 0, sd of 1

sim = (raw_sim - 0.5) / 0.5 # May need something more efficient for the full set?
# print(raw_sim[0,1])
# print(sim[0,1])

tensor = torch.from_numpy(sim)
tensor = tensor.double()
tensor = tensor.unsqueeze(dim = 1)

In [16]:
raw_sim[:,0,:]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False,  True, ..., False, False, False]])

In [7]:
tensor.shape # same as snps

torch.Size([100, 1, 72, 1263])

In [19]:
new_p = np.array([list(item) for item in param])
new_p = new_p[:, 2:7]
new_p = torch.from_numpy(np.array(new_p))
# print(new_p.shape)

train = torch.utils.data.TensorDataset(tensor, new_p)

trainloader = torch.utils.data.DataLoader(train, batch_size = 20,
                                          shuffle = True)

In [32]:
summary(model, input_size=(1, 72, 1263), batch_size=20)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [20, 6, 68, 1259]             156
         MaxPool2d-2           [20, 6, 17, 314]               0
            Conv2d-3          [20, 16, 13, 310]           2,416
         MaxPool2d-4            [20, 16, 3, 77]               0
            Linear-5                  [20, 120]         443,640
            Linear-6                   [20, 84]          10,164
            Linear-7                    [20, 5]             425
Total params: 456,801
Trainable params: 456,801
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 6.94
Forward/backward pass size (MB): 93.70
Params size (MB): 1.74
Estimated Total Size (MB): 102.38
----------------------------------------------------------------


In [30]:
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=3696, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)


In [8]:
# Main imports
import numpy as np
import torch
import time

# PyTorch shorthands
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

tic = time.perf_counter()

### data loader will need to be modified!! ###
# Currently using toy data set
with np.load('./toy_dat/sliced_data.npz') as data:
    param = data['param']
    raw_sim = data['sim']
    pos = data['pos']
	
# Normalize to mean of 0, sd of 1

sim = (raw_sim - 0.5) / 0.5 # May need something more efficient for the full set?
# print(raw_sim[0,1])
# print(sim[0,1])

tensor = torch.from_numpy(sim)
tensor = tensor.double()
tensor = tensor.unsqueeze(dim = 1)

new_p = np.array([list(item) for item in param])
new_p = new_p[:, 2:7]
new_p = torch.from_numpy(np.array(new_p))
# print(new_p.shape)

train = torch.utils.data.TensorDataset(tensor, new_p)

trainloader = torch.utils.data.DataLoader(train, batch_size = 20,
                                          shuffle = True)
# print(trainloader)
# tensor = tensor.unsqueeze(dim = 1)
# print(tensor.shape)
# print(tensor.shape)
#t1 = tensor[1,1]

# Build the net

class Net(nn.Module):
    # Define all the transformations we want here
    def __init__(self):
        # Inherit behaviour from PyTorch class
        super(Net, self).__init__()
        # Convolutional transforms
        # input to Conv2d should be a 4D tensor [batch_size, channel, l, w]
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(4, 4)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Regular neural net transforms after unwrap
        self.fc1 = nn.Linear(16 * 3 * 77, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    # Define all the steps for one full forward pass
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape)
        x = x.view(-1, 16 * 3 * 77)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Loss and optimizer functions

criterion = F.mse_loss
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.7)

n_epoch = 20

net = net.double()

# Load CUDA if present
# to load CUDA in colab, go to runtime->change runtime type and select gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: %s" % device)

net.to(device)

for epoch in range(n_epoch): # loope over dataset multiple times
    running_loss = 0.0
    count_b = 0
    for inputs, labels in trainloader:
        count_b += 1
        # data is a list of [inputs, labels]
        #inputs, labels = data[0], data[1] # note that moving tensors to the device is done _out of place_
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # print(f"labels as long: {labels.long()}")
        # print(inputs)
        #inputs = tensor[i,:]
        # zero parameter gradients
        optimizer.zero_grad()

        # forward -> backward -> optimize
        outputs = net(inputs)
        # print(f"output shape: {outputs.shape}" )
        # print(f"label shape: {labels.shape}" )
        # outputs = outputs.unsqueeze(dim=0)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if count_b % 5 == 4: # print every 20 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, count_b + 1, running_loss / 5))
            running_loss = 0.0

print('Finished Training')

toc = time.perf_counter()
print(f"Total time run: {(toc-tic)/60:0.4f} minutes")

### Where we save to needs to be modified!! ###
torch.save(net.state_dict(), "./test.pth")

Using device: cpu
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
[1,     5] loss: 56.977
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
[2,     5] loss: 18.966
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
[3,     5] loss: 252.761
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
[4,     5] loss: 225.861
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
[5,     5] loss: 235.039
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 72, 1263])
torch.Size([20, 1, 